In [1]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*


import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [2]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [3]:
#categorize training set
train_factor = train
train_factor['weather'] = train_factor['weather'].astype('category')
train_factor['holiday'] = train_factor['holiday'].astype('category')
train_factor['workingday'] = train_factor['workingday'].astype('category')
train_factor['season'] = train_factor['season'].astype('category')

#factorize test set
test_factor = test
test_factor['weather'] = test_factor['weather'].astype('category')
test_factor['holiday'] = test_factor['holiday'].astype('category')
test_factor['workingday'] = test_factor['workingday'].astype('category')
test_factor['season'] = test_factor['season'].astype('category')

In [4]:
#create day of week column and hour column then convert into category for both training and test set
train_factor['day']=0
train_factor['hour']=0
for i in range(len(train_factor)) :
	train_factor['day'][i] = train_factor['date'][i].weekday()
	train_factor['hour'][i] = train_factor['date'][i].hour

train_factor['day']=train_factor['day'].astype('category')
train_factor['hour']=train_factor['hour'].astype('category')

test_factor['day']=0
test_factor['hour']=0
for i in range(len(test_factor)) :
	test_factor['day'][i] = test_factor['date'][i].weekday()
	test_factor['hour'][i] = test_factor['date'][i].hour

test_factor['day']=test_factor['day'].astype('category')
test_factor['hour']=test_factor['hour'].astype('category')

In [5]:
#is a day more significant ?
byday=train_factor.groupby('day')
print(byday.describe())
print('Sunday seems to be an important predictor')

                 atemp       casual        count     humidity   registered  \
day                                                                          
0   count  1551.000000  1551.000000  1551.000000  1551.000000  1551.000000   
    mean     24.178507    29.843972   190.390716    62.508059   160.546744   
    std       8.413965    35.909764   180.943376    18.314346   160.634673   
    min       5.305000     0.000000     1.000000    15.000000     0.000000   
    25%      17.425000     4.000000    39.000000    48.000000    33.000000   
    50%      25.000000    16.000000   146.000000    63.000000   120.000000   
    75%      31.060000    44.000000   277.500000    77.000000   221.500000   
    max      42.425000   242.000000   968.000000   100.000000   857.000000   
1   count  1539.000000  1539.000000  1539.000000  1539.000000  1539.000000   
    mean     24.252898    22.979207   189.723847    63.662768   166.744639   
    std       8.500099    25.735518   186.719673    18.890279   

In [6]:
#create Sunday variable and convert to category 
train_factor['Sunday'] = 0 ;
train_factor.ix[train_factor['day']==6,'Sunday']=1

train_factor['Sunday']=train_factor['Sunday'].astype('category')

test_factor['Sunday'] = 0 ;
test_factor.ix[test_factor['day']==6,'Sunday']=1

test_factor['Sunday']=test_factor['Sunday'].astype('category')

In [7]:
train_factor['temp'].describe()

count    10886.00000
mean        20.23086
std          7.79159
min          0.82000
25%         13.94000
50%         20.50000
75%         26.24000
max         41.00000
Name: temp, dtype: float64

In [8]:
#create Hour variable and convert to category (4 category) for both training and test set
train_factor['daypart'] = 0
for i in range(len(train_factor)) :
	currentHour = train_factor['hour'][i]
	if (currentHour>=16 and currentHour<21 ):
		train_factor['daypart'][i] = 3
	elif (currentHour>=11 and currentHour<16):
		train_factor['daypart'][i] = 2
	elif (currentHour>=4 and currentHour<11) :
		train_factor['daypart'][i] = 1

train_factor['daypart']=train_factor['daypart'].astype('category')

test_factor['daypart'] = 0
for i in range(len(test_factor)) :
	currentHour = test_factor['hour'][i]
	if (currentHour>=16 and currentHour<21 ):
		test_factor['daypart'][i] = 3
	elif (currentHour>=11 and currentHour<16):
		test_factor['daypart'][i] = 2
	elif (currentHour>=4 and currentHour<11) :
		test_factor['daypart'][i] = 1

test_factor['daypart']=test_factor['daypart'].astype('category')

In [9]:
#create ahot () variable and convert to category (4 category) for both training and test set
train_factor['ahot'] = 4
for i in range(len(train_factor)) :
	currentTemp = train_factor['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train_factor['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train_factor['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train_factor['ahot'][i] = 1

train_factor['ahot']=train_factor['ahot'].astype('category')

test_factor['ahot'] = 4
for i in range(len(test_factor)) :
	currentTemp = test_factor['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test_factor['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test_factor['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test_factor['ahot'][i] = 1

test_factor['ahot']=test_factor['ahot'].astype('category')

In [10]:
train_factor['temp'].describe()

count    10886.00000
mean        20.23086
std          7.79159
min          0.82000
25%         13.94000
50%         20.50000
75%         26.24000
max         41.00000
Name: temp, dtype: float64

In [11]:
#create ahot () variable and convert to category (4 category) for both training and test set
train_factor['hot'] = 4
for i in range(len(train_factor)) :
	currentTemp = train_factor['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train_factor['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train_factor['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train_factor['hot'][i] = 1

train_factor['hot']=train_factor['hot'].astype('category')

test_factor['hot'] = 4
for i in range(len(test_factor)) :
	currentTemp = test_factor['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test_factor['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test_factor['hot'][i] = 2
	elif (currentTemp<13.94) :
		test_factor['hot'][i] = 1

test_factor['hot']=test_factor['hot'].astype('category')

In [12]:
train_factor[train_factor['ahot']==4]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,day,hour,Sunday,daypart,ahot,hot
datetime,,,,,,,,,,,,,,,,,,
2011-02-17 14:00:00,1,0,1,1,24.60,31.060,31,15.0013,26,55,81,2011-02-17 14:00:00,3,14,0,2,4,3
2011-02-18 13:00:00,1,0,1,1,24.60,31.060,43,12.9980,45,131,176,2011-02-18 13:00:00,4,13,0,2,4,3
2011-02-18 14:00:00,1,0,1,1,27.06,31.060,36,19.9995,73,118,191,2011-02-18 14:00:00,4,14,0,2,4,4
2011-02-18 15:00:00,1,0,1,1,27.06,31.060,36,22.0028,55,117,172,2011-02-18 15:00:00,4,15,0,2,4,4
2011-02-18 16:00:00,1,0,1,1,27.06,31.060,36,19.0012,68,164,232,2011-02-18 16:00:00,4,16,0,3,4,4
2011-02-18 17:00:00,1,0,1,1,27.06,31.060,34,23.9994,52,275,327,2011-02-18 17:00:00,4,17,0,3,4,4
2011-02-18 18:00:00,1,0,1,1,26.24,31.060,33,22.0028,29,195,224,2011-02-18 18:00:00,4,18,0,3,4,4
2011-02-18 19:00:00,1,0,1,1,25.42,31.060,29,39.0007,16,146,162,2011-02-18 19:00:00,4,19,0,3,4,3
2011-02-18 20:00:00,1,0,1,1,24.60,31.060,31,12.9980,19,105,124,2011-02-18 20:00:00,4,20,0,3,4,3


In [70]:
#linear regression weekday/weekend
formula = "count ~ season + weather + temp +  windspeed + humidity  + hour + ahot + hot"

train_factor_weekday=train_factor[(train_factor['day']!=5)]
train_factor_weekday=train_factor_weekday[(train_factor_weekday['day']!=6)]
regression_weekday=smf.glm(formula,data=train_factor_weekday,family=sm.families.Poisson(link=sm.families.links.log))
regression_weekday=regression_weekday.fit()
print(regression_weekday.summary2())

test_factor_weekday=test_factor[(test_factor['day']!=5)]
test_factor_weekday=test_factor_weekday[(test_factor_weekday['day']!=6)]

formula = "count ~ season + weather + temp +  windspeed + humidity + hour + Sunday + ahot + hot"
train_factor_weekend=train_factor[(train_factor['day']!=0)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=1)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=2)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=3)]
train_factor_weekend=train_factor_weekend[(train_factor_weekend['day']!=4)]
regression_weekend=smf.glm(formula,data=train_factor_weekend,family=sm.families.Poisson(link=sm.families.links.log))
regression_weekend=regression_weekend.fit()
print(regression_weekend.summary2())

test_factor_weekend=test_factor[(test_factor['day']!=0)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=1)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=2)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=3)]
test_factor_weekend=test_factor_weekend[(test_factor_weekend['day']!=4)]

               Results: Generalized linear model
Model:              GLM              AIC:            230781.5468
Link Function:      log              BIC:            112639.1827
Dependent Variable: count            Log-Likelihood: -1.1535e+05
Date:               2016-05-23 16:23 LL-Null:        -6.7370e+05
No. Observations:   7723             Deviance:       1.8143e+05 
Df Model:           38               Pearson chi2:   1.84e+05   
Df Residuals:       7684             Scale:          1.0000     
Method:             IRLS                                        
----------------------------------------------------------------
                Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
----------------------------------------------------------------
Intercept       3.1592   0.0109  290.5565 0.0000  3.1379  3.1805
season[T.2]     0.2658   0.0033   80.0841 0.0000  0.2593  0.2723
season[T.3]     0.2533   0.0040   63.2460 0.0000  0.2455  0.2612
season[T.4]     0.4287   0.0029  147.0680

In [60]:
train_factor.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,date,day,hour,Sunday,daypart,ahot,hot
datetime,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011-01-01 00:00:00,5,0,0,0,1,1
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011-01-01 01:00:00,5,1,0,0,1,1
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011-01-01 02:00:00,5,2,0,0,1,1
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011-01-01 03:00:00,5,3,0,0,1,1
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011-01-01 04:00:00,5,4,0,1,1,1


In [41]:
#linear regression all week
formula = "count ~season +  holiday + workingday + weather + day + ahot + hot + hour + Sunday"
regression_allWeek = smf.glm(formula,data=train_factor,family=sm.families.Poisson(link=sm.families.links.log))
regression_allWeek=regression_allWeek.fit()
print(regression_allWeek.summary2())



                Results: Generalized linear model
Model:               GLM              AIC:            530869.7701
Link Function:       log              BIC:            360348.6407
Dependent Variable:  count            Log-Likelihood: -2.6539e+05
Date:                2016-05-23 16:13 LL-Null:        -9.3511e+05
No. Observations:    10886            Deviance:       4.6114e+05 
Df Model:            42               Pearson chi2:   4.69e+05   
Df Residuals:        10843            Scale:          1.0000     
Method:              IRLS                                        
-----------------------------------------------------------------
                 Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept        2.6261   0.0053  496.0372 0.0000  2.6157  2.6365
season[T.2]      0.2591   0.0028   93.6136 0.0000  0.2537  0.2645
season[T.3]      0.2267   0.0032   71.4269 0.0000  0.2204  0.2329
season[T.4]      0.4044   

In [71]:
#prediction weekday/weekend
prediction_weekday=regression_weekday.predict(test_factor_weekday)
prediction_weekday[prediction_weekday<0]=0 # bike demand is positive

prediction_weekend=regression_weekend.predict(test_factor_weekend)
prediction_weekend[prediction_weekend<0]=0 # bike demand is positive



In [50]:
#prediction all week
prediction_allWeek=regression_allWeek.predict(test_factor)
prediction_allWeek[prediction_allWeek<0]=0 # bike demand is positive


In [72]:
#convert to dataframe
prediction_weekend=pd.DataFrame(prediction_weekend)
prediction_weekend.columns=['count']
prediction_weekend["datetime"]=test_factor_weekend.index.values
prediction_weekend = prediction_weekend.set_index("datetime")

prediction_weekday=pd.DataFrame(prediction_weekday)
prediction_weekday.columns=['count']
prediction_weekday["datetime"]=test_factor_weekday.index.values
prediction_weekday = prediction_weekday.set_index("datetime")

prediction_allWeek=pd.DataFrame(prediction_allWeek)
prediction_allWeek.columns=['count']
prediction_allWeek["datetime"]=test_factor.index.values
prediction_allWeek=prediction_allWeek.set_index("datetime")

In [68]:
#write prediction for weekend/weekday
prediction=prediction_weekend;
prediction=prediction.append(prediction_weekday)
prediction.sort_index()
print(prediction)

                          count
datetime                       
2011-01-22 00:00:00   54.769095
2011-01-22 01:00:00   39.561529
2011-01-22 02:00:00   31.829683
2011-01-22 03:00:00   15.190133
2011-01-22 04:00:00    4.886956
2011-01-22 06:00:00   10.872071
2011-01-22 07:00:00   24.858083
2011-01-22 08:00:00   57.481504
2011-01-22 09:00:00   99.410503
2011-01-22 10:00:00  142.461412
2011-01-22 11:00:00  173.801737
2011-01-22 12:00:00  198.761169
2011-01-22 13:00:00  219.879216
2011-01-22 14:00:00  204.894428
2011-01-22 15:00:00  215.464981
2011-01-22 16:00:00  192.600001
2011-01-22 17:00:00  195.880192
2011-01-22 18:00:00  150.205492
2011-01-22 19:00:00  131.399201
2011-01-22 20:00:00   97.585669
2011-01-22 21:00:00   83.781304
2011-01-22 22:00:00   71.912571
2011-01-22 23:00:00   52.615702
2011-01-23 00:00:00   50.658609
2011-01-23 01:00:00   39.844751
2011-01-23 02:00:00   29.305940
2011-01-23 03:00:00   14.164767
2011-01-23 05:00:00    4.767329
2011-01-23 06:00:00    9.870843
2011-01-

In [73]:
#write the submission
prediction.to_csv("Résultats/regressionPoissonFactorWeekend_Weekday.csv")


In [46]:
#write prediction for all week
prediction=prediction_allWeek;
prediction.sort_index()
print(prediction)

                          count
datetime                       
2011-01-20 00:00:00   30.871374
2011-01-20 01:00:00   19.447226
2011-01-20 02:00:00   13.096693
2011-01-20 03:00:00    6.703917
2011-01-20 04:00:00    3.738035
2011-01-20 05:00:00   11.700880
2011-01-20 06:00:00   45.979317
2011-01-20 07:00:00  126.929356
2011-01-20 08:00:00  210.002806
2011-01-20 09:00:00  114.818386
2011-01-20 10:00:00   95.218718
2011-01-20 11:00:00  102.177907
2011-01-20 12:00:00  154.215434
2011-01-20 13:00:00  120.011051
2011-01-20 14:00:00  112.600744
2011-01-20 15:00:00  117.611678
2011-01-20 16:00:00  145.681509
2011-01-20 17:00:00  222.194816
2011-01-20 18:00:00  206.123111
2011-01-20 19:00:00  164.740043
2011-01-20 20:00:00  111.877244
2011-01-20 21:00:00   86.067488
2011-01-20 22:00:00   67.379285
2011-01-20 23:00:00   46.558842
2011-01-21 00:00:00   28.615961
2011-01-21 01:00:00   18.026444
2011-01-21 02:00:00    7.598175
2011-01-21 03:00:00    3.889343
2011-01-21 04:00:00    3.464940
2011-01-

In [47]:
#write the submission
prediction.to_csv("Résultats/regressionPoissonFactorAll_Week.csv")


In [23]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [24]:
lr=LogisticRegression()

In [25]:
#algorithme de sélection de var.
from sklearn.feature_selection import RFE
selecteur=RFE(estimator=lr)

In [26]:
train_factor.()

SyntaxError: invalid syntax (<ipython-input-26-1d249e5f4f84>, line 1)

In [ ]:
data_X=train_factor.drop(["casual","registered","count","date"],1)
data_Y=train_factor['count']
data_Y=data_Y.as_matrix()
data_X=data_X.as_matrix()
data_X

In [ ]:
sol=selecteur.fit(data_X,data_Y)

In [ ]:
sol.n_features_

In [ ]:
print(train_factor.drop(["casual","registered","count","date"],1).head())
print(sol.support_)
print(sol.ranking_)